In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score , mean_squared_error
from sklearn.model_selection import  GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/kaggle/input/cleaned-data-uber/cleaned_data.csv')

In [3]:
df.head()

Unnamed: 0 Car Condition Weather  Traffic Condition  fare_amount  \
0           2           Bad  stormy  Congested Traffic          5.7   
1           3     Very Good  stormy       Flow Traffic          7.7   
2           4           Bad  stormy  Congested Traffic          5.3   
3           5     Excellent   sunny  Congested Traffic         12.1   
4           6     Excellent   windy       Flow Traffic          7.5   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0         -1.291242         0.711418          -1.291391          0.711231   
1         -1.291319         0.710927          -1.291396          0.711363   
2         -1.290987         0.711536          -1.290787          0.711811   
3         -1.291560         0.710901          -1.291071          0.711365   
4         -1.291195         0.711251          -1.291086          0.711481   

   passenger_count  ...   bearing  pickup_hour  pickup_day  pickup_month  \
0                2  ...  2.599961            0          18             8   
1                1  ...  0.133905            4          21             4   
2                1  ... -0.502703            7           9             3   
3                1  ... -0.674251            9           6             1   
4                1  ... -0.342260           20          20            11   

   pickup_year  pickup_weekday  day_of_week     month  is_weekend  \
0         2011               3     Thursday    August       False   
1         2012               5     Saturday     April        True   
2         2010               1      Tuesday     March       False   
3         2011               3     Thursday   January       False   
4         2012               1      Tuesday  November       False   

   manhattan_distance  
0            0.000335  
1            0.000513  
2            0.000475  
3            0.000954  
4            0.000338  

[5 rows x 26 columns]

In [4]:
df.drop(['Unnamed: 0','is_weekend','manhattan_distance'],axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341462 entries, 0 to 341461
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Car Condition      341462 non-null  object 
 1   Weather            341462 non-null  object 
 2   Traffic Condition  341462 non-null  object 
 3   fare_amount        341462 non-null  float64
 4   pickup_longitude   341462 non-null  float64
 5   pickup_latitude    341462 non-null  float64
 6   dropoff_longitude  341462 non-null  float64
 7   dropoff_latitude   341462 non-null  float64
 8   passenger_count    341462 non-null  int64  
 9   jfk_dist           341462 non-null  float64
 10  ewr_dist           341462 non-null  float64
 11  lga_dist           341462 non-null  float64
 12  sol_dist           341462 non-null  float64
 13  nyc_dist           341462 non-null  float64
 14  distance           341462 non-null  float64
 15  bearing            341462 non-null  float64
 16  pi

In [6]:
label_encoders = {}
categorical_cols = df.select_dtypes('object')
for col in categorical_cols:
    lb = LabelEncoder()
    df[col] = lb.fit_transform(df[col])
    label_encoders[col] = lb

In [7]:
df.head()

Car Condition  Weather  Traffic Condition  fare_amount  pickup_longitude  \
0              0        2                  0          5.7         -1.291242   
1              3        2                  2          7.7         -1.291319   
2              0        2                  0          5.3         -1.290987   
3              1        3                  0         12.1         -1.291560   
4              1        4                  2          7.5         -1.291195   

   pickup_latitude  dropoff_longitude  dropoff_latitude  passenger_count  \
0         0.711418          -1.291391          0.711231                2   
1         0.710927          -1.291396          0.711363                1   
2         0.711536          -1.290787          0.711811                1   
3         0.710901          -1.291071          0.711365                1   
4         0.711251          -1.291086          0.711481                1   

    jfk_dist  ...   nyc_dist  distance   bearing  pickup_hour  pickup_day  \
0  43.597686  ...   9.847344  1.389525  2.599961            0          18   
1  42.642965  ...   7.703421  2.799270  0.133905            4          21   
2  43.329953  ...  15.600745  1.999157 -0.502703            7           9   
3  42.335622  ...   7.649541  3.787239 -0.674251            9           6   
4  42.563234  ...  10.993150  1.555807 -0.342260           20          20   

   pickup_month  pickup_year  pickup_weekday  day_of_week  month  
0             8         2011               3            4      1  
1             4         2012               5            2      0  
2             3         2010               1            5      7  
3             1         2011               3            4      4  
4            11         2012               1            5      9  

[5 rows x 23 columns]

In [8]:
label_encoders

{'Car Condition': LabelEncoder(),
 'Weather': LabelEncoder(),
 'Traffic Condition': LabelEncoder(),
 'day_of_week': LabelEncoder(),
 'month': LabelEncoder()}

## split data

In [9]:
x = df.drop(columns=["fare_amount"])  
y = df["fare_amount"]  

In [10]:
# Split data into train and test 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(273169, 22)
(68293, 22)
(273169,)
(68293,)


In [12]:
# normalizing train set (standardization)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
# # Linear Regression
# linear_reg = LinearRegression()

# # Ridge Regression
# ridge_reg = Ridge()
# ridge_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

# # Lasso Regression
# lasso_reg = Lasso()
# lasso_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

# # Decision Tree Regression
# tree_reg = DecisionTreeRegressor(random_state=42)
# tree_params = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}

# # Random Forest Regression
# forest_reg = RandomForestRegressor(random_state=42)
# forest_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

# # Gradient Boosting Regression
# gradient_reg = GradientBoostingRegressor(random_state=42)
# gradient_params = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}

# # Support Vector Regression (SVR)
# svr_reg = SVR()
# svr_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

In [ ]:
# # Function to perform GridSearchCV
# def tune_model(model, params, x_train, y_train):
#     grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
#     grid_search.fit(x_train, y_train)
#     return grid_search.best_estimator_, grid_search.best_params_

# # Tune models
# best_ridge, ridge_best_params = tune_model(ridge_reg, ridge_params, x_train, y_train)
# best_lasso, lasso_best_params = tune_model(lasso_reg, lasso_params, x_train, y_train)
# best_tree, tree_best_params = tune_model(tree_reg, tree_params, x_train, y_train)
# best_forest, forest_best_params = tune_model(forest_reg, forest_params, x_train, y_train)
# best_gradient, gradient_best_params = tune_model(gradient_reg, gradient_params, x_train, y_train)
# best_svr, svr_best_params = tune_model(svr_reg, svr_params, x_train, y_train)

In [16]:
# Initialize the model
linear_reg = LinearRegression()

# Train the model
linear_reg.fit(x_train, y_train)

# Predict on the test set
y_pred = linear_reg.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Linear Regression - MSE: {mse}, R2 Score: {r2}")


Linear Regression - MSE: 4.27898338223272, R2 Score: 0.6380353702494281


## ridge

In [20]:
# Initialize the model
ridge_reg = Ridge()

# Define hyperparameter grid
ridge_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Perform GridSearchCV
ridge_grid = GridSearchCV(ridge_reg, ridge_params, scoring='r2', cv=5)
ridge_grid.fit(x_train, y_train)

# Best model and parameters
best_ridge = ridge_grid.best_estimator_
print(f"Best Ridge Parameters: {ridge_grid.best_params_}")

Best Ridge Parameters: {'alpha': 1}


In [23]:
# Predict on the test set
y_pred = best_ridge.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Ridge Regression - MSE: {mse}, R2 Score: {r2}")

# Cross-validation
cv_scores = cross_val_score(best_ridge, x, y, scoring='neg_mean_squared_error', cv=5)
cv_mse = -cv_scores.mean()
print(f"Ridge Regression - CV MSE: {cv_mse}")

Ridge Regression - MSE: 4.279075344812848, R2 Score: 0.6380275910181732
Ridge Regression - CV MSE: 4.353589732475296


## lasso

In [25]:
# Initialize the model
lasso_reg = Lasso()

# Define hyperparameter grid
lasso_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Perform GridSearchCV
lasso_grid = GridSearchCV(lasso_reg, lasso_params, scoring='r2', cv=5)
lasso_grid.fit(x_train, y_train)

# Best model and parameters
best_lasso = lasso_grid.best_estimator_
print(f"Best Lasso Parameters: {lasso_grid.best_params_}")

Best Lasso Parameters: {'alpha': 0.01}


In [26]:
# Predict on the test set
y_pred = best_lasso.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Lasso Regression - MSE: {mse}, R2 Score: {r2}")

# Cross-validation
cv_scores = cross_val_score(best_lasso, x, y, scoring='neg_mean_squared_error', cv=5)
cv_mse = -cv_scores.mean()
print(f"Lasso Regression - CV MSE: {cv_mse}")

Lasso Regression - MSE: 4.32332105407989, R2 Score: 0.6342847903708522
Lasso Regression - CV MSE: 4.369724785701015


## decision tree

In [28]:
# Initialize the model
tree_reg = DecisionTreeRegressor(random_state=42)

# Define hyperparameter grid
tree_params = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}

# Perform GridSearchCV
tree_grid = GridSearchCV(tree_reg, tree_params, scoring='r2', cv=5)
tree_grid.fit(x_train, y_train)

# Best model and parameters
best_tree = tree_grid.best_estimator_
print(f"Best Decision Tree Parameters: {tree_grid.best_params_}")

Best Decision Tree Parameters: {'max_depth': 10, 'min_samples_split': 10}


In [29]:
# Predict on the test set
y_pred = best_tree.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Decision Tree Regression - MSE: {mse}, R2 Score: {r2}")

# Cross-validation
cv_scores = cross_val_score(best_tree, x, y, scoring='neg_mean_squared_error', cv=5)
cv_mse = -cv_scores.mean()
print(f"Decision Tree Regression - CV MSE: {cv_mse}")

Decision Tree Regression - MSE: 3.6522401047927526, R2 Score: 0.6910523787541085
Decision Tree Regression - CV MSE: 3.6607449980678104


## random forest

In [37]:
# Initialize the model
forest_reg = RandomForestRegressor(random_state=42, n_jobs=-1)

# Define hyperparameter grid
forest_params = {'n_estimators': [50], 'max_depth': [ 10,15, 20]}

# Perform GridSearchCV
forest_grid = GridSearchCV(forest_reg, forest_params, scoring='neg_mean_squared_error', cv=5)
forest_grid.fit(x_train, y_train)

# Best model and parameters
best_forest = forest_grid.best_estimator_
print(f"Best Random Forest Parameters: {forest_grid.best_params_}")

Best Random Forest Parameters: {'max_depth': 20, 'n_estimators': 50}


In [38]:
# Predict on the test set
y_pred = best_forest.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Random Forest Regression - MSE: {mse}, R2 Score: {r2}")

# Cross-validation
cv_scores = cross_val_score(best_forest, x, y, scoring='neg_mean_squared_error', cv=5)
cv_mse = -cv_scores.mean()
print(f"Random Forest Regression - CV MSE: {cv_mse}")

Random Forest Regression - MSE: 3.0444661226213587, R2 Score: 0.7424647505202988
Random Forest Regression - CV MSE: 3.0351435300156817


## XGBoost

In [14]:
def objective_xgboost(trial):
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42,
        'n_jobs': -1
    }

    # Initialize the XGBoost model
    model = xgb.XGBRegressor(**params)

    # Evaluate the model using cross-validation
    scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
    mse = -scores.mean()  # Convert to positive MSE

    return mse

In [20]:
# Create an Optuna study
study_xgboost = optuna.create_study(direction='minimize')  # Minimize MSE

# Optimize the objective function
study_xgboost.optimize(objective_xgboost, n_trials=20)  # Number of trials

# Print the best hyperparameters
print("Best XGBoost Hyperparameters:", study_xgboost.best_params)
print("Best XGBoost MSE:", study_xgboost.best_value)

[I 2025-03-05 16:59:33,235] A new study created in memory with name: no-name-d5f947fe-3fd9-4a2e-b1c2-be73544696bf
[I 2025-03-05 17:00:20,986] Trial 0 finished with value: 3.0918646453056646 and parameters: {'n_estimators': 387, 'max_depth': 12, 'learning_rate': 0.2352050612667415, 'subsample': 0.7101353220650287, 'colsample_bytree': 0.6519299252277029, 'gamma': 0.8822764724305616, 'reg_alpha': 0.3705442607129136, 'reg_lambda': 0.8378502875088384}. Best is trial 0 with value: 3.0918646453056646.
[I 2025-03-05 17:01:18,233] Trial 1 finished with value: 2.691093203435907 and parameters: {'n_estimators': 415, 'max_depth': 10, 'learning_rate': 0.06738787134640767, 'subsample': 0.8189156182435493, 'colsample_bytree': 0.5368599725680014, 'gamma': 0.6497305383227495, 'reg_alpha': 0.8436025381182491, 'reg_lambda': 0.586062923965153}. Best is trial 1 with value: 2.691093203435907.
[I 2025-03-05 17:01:34,305] Trial 2 finished with value: 2.803007914800761 and parameters: {'n_estimators': 384, 'ma

Best XGBoost Hyperparameters: {'n_estimators': 415, 'max_depth': 10, 'learning_rate': 0.06738787134640767, 'subsample': 0.8189156182435493, 'colsample_bytree': 0.5368599725680014, 'gamma': 0.6497305383227495, 'reg_alpha': 0.8436025381182491, 'reg_lambda': 0.586062923965153}
Best XGBoost MSE: 2.691093203435907


In [21]:
# Extract the best hyperparameters
best_params_xgboost = study_xgboost.best_params

# Initialize the best XGBoost model
best_xgboost = xgb.XGBRegressor(**best_params_xgboost, random_state=42, n_jobs=-1)

# Train the model on the full training set
best_xgboost.fit(x_train, y_train)

# Predict on the test set
y_pred_xgboost = best_xgboost.predict(x_test)

# Evaluate the model
mse_xgboost = mean_squared_error(y_test, y_pred_xgboost)
r2_xgboost = r2_score(y_test, y_pred_xgboost)
print(f"XGBoost Test MSE: {mse_xgboost}")
print(f"XGBoost Test R2 Score: {r2_xgboost}")

XGBoost Test MSE: 2.6674782308319247
XGBoost Test R2 Score: 0.7743546342806816


## gradient boost

In [ ]:
# Initialize the model
gradient_reg = GradientBoostingRegressor(random_state=42)

# Define hyperparameter grid
gradient_params = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}

# Perform GridSearchCV
gradient_grid = GridSearchCV(gradient_reg, gradient_params, scoring='r2', cv=3)
gradient_grid.fit(x_train, y_train)

# Best model and parameters
best_gradient = gradient_grid.best_estimator_
print(f"Best Gradient Boosting Parameters: {gradient_grid.best_params_}")

In [ ]:
# Predict on the test set
y_pred = best_gradient.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Gradient Boosting Regression - MSE: {mse}, R2 Score: {r2}")

# Cross-validation
cv_scores = cross_val_score(best_gradient, x, y, scoring='neg_mean_squared_error', cv=5)
cv_mse = -cv_scores.mean()
print(f"Gradient Boosting Regression - CV MSE: {cv_mse}")

## SVR

In [ ]:
# # Initialize the model
# svr_reg = SVR()

# # Define hyperparameter grid
# svr_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}

# # Perform GridSearchCV
# svr_grid = GridSearchCV(svr_reg, svr_params, scoring='r2', cv=5)
# svr_grid.fit(x_train, y_train)

# # Best model and parameters
# best_svr = svr_grid.best_estimator_
# print(f"Best SVR Parameters: {svr_grid.best_params_}")

In [ ]:
# # Predict on the test set
# y_pred = best_svr.predict(x_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f"SVR - MSE: {mse}, R2 Score: {r2}")

# # Cross-validation
# cv_scores = cross_val_score(best_svr, x, y, scoring='r2', cv=5)
# cv_mse = -cv_scores.mean()
# print(f"SVR - CV MSE: {cv_mse}")

In [15]:
# def objective_gb(trial):
#     # Define hyperparameters to tune
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
#         'random_state': 42
#     }

#     # Initialize the Gradient Boosting model
#     model = GradientBoostingRegressor(**params)

#     # Evaluate the model using cross-validation
#     scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
#     mse = -scores.mean()  # Convert to positive MSE

#     return mse

In [ ]:
# # Create an Optuna study
# study_gb = optuna.create_study(direction='minimize')

# # Optimize the objective function
# study_gb.optimize(objective_gb, n_trials=20)

# # Print the best hyperparameters
# print("Best Gradient Boosting Hyperparameters:", study_gb.best_params)
# print("Best Gradient Boosting MSE:", study_gb.best_value)

[I 2025-03-05 20:13:52,726] A new study created in memory with name: no-name-4c19e40b-10cf-45e4-83e0-cd26fcdeb768


In [ ]:
# # Extract the best hyperparameters
# best_params_gb = study_gb.best_params

# # Initialize the best Gradient Boosting model
# best_gb = GradientBoostingRegressor(**best_params_gb, random_state=42)

# # Train the model on the full training set
# best_gb.fit(x_train, y_train)

# # Predict on the test set
# y_pred_gb = best_gb.predict(x_test)

# # Evaluate the model
# mse_gb = mean_squared_error(y_test, y_pred_gb)
# r2_gb = r2_score(y_test, y_pred_gb)
# print(f"Gradient Boosting Test MSE: {mse_gb}")
# print(f"Gradient Boosting Test R2 Score: {r2_gb}")

In [20]:
# def objective_svr(trial):
#     # Define hyperparameters to tune
#     params = {
#         'C': trial.suggest_float('C', 0.1, 10),
#         'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf']),
#         'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
#         'epsilon': trial.suggest_float('epsilon', 0.01, 0.2)
#     }

#     # Initialize the SVR model
#     model = SVR(**params)

#     # Evaluate the model using cross-validation
#     scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
#     mse = -scores.mean()  # Convert to positive MSE

#     return mse

In [ ]:
# # Create an Optuna study
# study_svr = optuna.create_study(direction='minimize')

# # Optimize the objective function
# study_svr.optimize(objective_svr, n_trials=20)

# # Print the best hyperparameters
# print("Best SVR Hyperparameters:", study_svr.best_params)
# print("Best SVR MSE:", study_svr.best_value)

[I 2025-03-05 18:39:39,935] A new study created in memory with name: no-name-4850ce9f-e50d-4c82-8e70-ac32bad0f0b3


In [ ]:
# # Extract the best hyperparameters
# best_params_svr = study_svr.best_params

# # Initialize the best SVR model
# best_svr = SVR(**best_params_svr)

# # Train the model on the full training set
# best_svr.fit(x_train, y_train)

# # Predict on the test set
# y_pred_svr = best_svr.predict(x_test)

# # Evaluate the model
# mse_svr = mean_squared_error(y_test, y_pred_svr)
# r2_svr = r2_score(y_test, y_pred_svr)
# print(f"SVR Test MSE: {mse_svr}")
# print(f"SVR Test R2 Score: {r2_svr}")

In [18]:
# def objective_svr(trial):
#     # Define hyperparameters to tune
#     params = {
#         'C': trial.suggest_float('C', 0.1, 10, log=True),  # Log scale for better exploration
#         'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf']),  # Use 'linear' for speed
#         'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),  # Avoid tuning gamma for 'linear'
#         'epsilon': trial.suggest_float('epsilon', 0.01, 0.2)
#     }

#     # Initialize the SVR model
#     model = SVR(**params)

#     # Evaluate the model using cross-validation
#     scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=3)  # Fewer folds for speed
#     mse = -scores.mean()  # Convert to positive MSE

#     # Prune unpromising trials
#     if trial.should_prune():
#         raise optuna.TrialPruned()

#     return mse

In [ ]:
# # Create an Optuna study with TPE sampler and median pruner
# study_svr = optuna.create_study(
#     direction='minimize',
#     sampler=TPESampler(),  # Tree-structured Parzen Estimator for efficient search
#     pruner=MedianPruner()  # Prune unpromising trials
# )

# # Optimize the objective function
# study_svr.optimize(objective_svr, n_trials=30)  # Fewer trials for speed

# # Print the best hyperparameters
# print("Best SVR Hyperparameters:", study_svr.best_params)
# print("Best SVR MSE:", study_svr.best_value)

[I 2025-03-05 18:49:30,331] A new study created in memory with name: no-name-4be5978e-b026-497e-8c40-ba4961b477f4


In [ ]:
# # Extract the best hyperparameters
# best_params_svr = study_svr.best_params

# # Initialize the best SVR model
# best_svr = SVR(**best_params_svr)

# # Train the model on the full training set
# best_svr.fit(X_train, y_train)

# # Predict on the test set
# y_pred_svr = best_svr.predict(X_test)

# # Evaluate the model
# mse_svr = mean_squared_error(y_test, y_pred_svr)
# r2_svr = r2_score(y_test, y_pred_svr)
# print(f"SVR Test MSE: {mse_svr}")
# print(f"SVR Test R2 Score: {r2_svr}")